In [153]:
import re
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [154]:
#set parameters
rsrpThr = -110
minDist = 0.02
minPts = 5

In [155]:
#directories
inputFile = r"c:/Work/Operators/scanner/TaoYuan.xlsx"

In [156]:
#read excel
scanner = pd.read_excel(inputFile, sheet_name="Series Formatted Data")

In [157]:
#change column names: only keep  PCI, RSRP and the corresponding ARFCN
oldColName = scanner.columns
pattern = r"(NR_Scan_|_SortedBy_RSRP_for_NRARFCN_|_0|SS_)"
newColName = {}

#create oldColName to newColName mapping dictionary
for colName in oldColName:
    temp = re.sub(pattern, "", colName)
    newColName[colName] = temp

#rename coluimn names
scanner = scanner.rename(columns = newColName)

In [158]:
#wide to long
scannerL = pd.wide_to_long(scanner, ['PCI', 'RSRP'], j = "ARFCN", i=["Longitude", "Latitude"])
#scannerL.head()

In [159]:
#remove NaN
scannerL = scannerL.dropna(axis=0,how="any")
#scannerL.describe()

In [160]:
#set lon, lat, ARFCN to column
scannerL = scannerL.reset_index(['Longitude', 'Latitude', 'ARFCN'])
#scannerL.head()

In [161]:
scannerL['ARFCN'].unique()

array([631000, 431570, 625324, 636000], dtype=int64)

In [162]:
scannerLfRsrp = scannerL[scannerL['RSRP'] > rsrpThr ]
#scannerLfRsrp.describe()

In [163]:
uPCI = scannerL.groupby('ARFCN').PCI.nunique()
uPCI = pd.DataFrame(uPCI).reset_index()
uPCI = uPCI.rename(columns = {'ARFCN': 'ARFCN', 'PCI': 'uPCI'})
uPCI

,ARFCN,uPCI
0,431570,15
1,625324,81
2,631000,137
3,636000,44


In [164]:
uPCIfRsrp = scannerLfRsrp.groupby('ARFCN').PCI.nunique()
uPCIfRsrp = pd.DataFrame(uPCIfRsrp).reset_index()
uPCIfRsrp = uPCIfRsrp.rename(columns = {'ARFCN': 'ARFCN', 'PCI': 'uPCIfRSRP'})
uPCIfRsrp

,ARFCN,uPCIfRSRP
0,431570,15
1,625324,75
2,631000,123
3,636000,5


In [165]:
tList = list()

#do DBSCAN on operaterBand and PCI level
for oprBand in scannerLfRsrp['ARFCN'].unique():
    #get unique PCI on ARFCN level
    for tpci in scannerLfRsrp[scannerLfRsrp['ARFCN'] == oprBand]['PCI'].unique():
        temp = scannerLfRsrp[(scannerLfRsrp['ARFCN'] == oprBand) & (scannerLfRsrp['PCI'] == tpci)]
        #DBSCAN
        db = DBSCAN(eps = minDist, min_samples=minPts).fit(temp[['Latitude', 'Longitude']])
        tdf = pd.concat([temp.reset_index(drop=True), pd.DataFrame(db.labels_, columns= ["clusterID"])], axis=1)
        tList.append(tdf)
        
#Merge all the list into one dataframe
tMerge = pd.concat(tList)            

#remove non clustered data points where clusterID = -1
tMergeCluster = tMerge[tMerge['clusterID'] > -1]


In [166]:
tMergeCluster.describe()

,Longitude,Latitude,ARFCN,Distance,Message,PCI,RSRP,clusterID
count,20381.000000,20381.000000,20381.000000,20381.000000,20381.000000,20381.000000,20381.000000,20381.000000
mean,121.272303,24.984075,618420.052598,14251.421147,243316.375350,220.062607,-91.456359,0.156715
std,0.043956,0.036154,43069.725562,8425.526152,156586.194237,138.721242,13.080951,0.363540
min,121.062810,24.818860,431570.000000,0.217942,19547.000000,0.000000,-109.980000,0.000000
25%,121.228960,24.958440,625324.000000,7810.861816,121194.000000,90.000000,-102.900000,0.000000
50%,121.294140,24.987810,625324.000000,13428.709961,227316.000000,220.000000,-93.300000,0.000000
75%,121.309790,25.001360,631000.000000,20341.585938,311872.000000,344.000000,-81.330000,0.000000
max,121.406160,25.110660,636000.000000,41313.558594,598156.000000,848.000000,-43.600000,1.000000


In [167]:
#tMergeCluster['clusterID'] =  tMergeCluster['clusterID'] + 1

In [194]:
#calculate cell count by operatorBand 
#cellCount =  tMergeCluster.groupby(['ARFCN', 'PCI']).clusterID.nunique().reset_index().groupby('ARFCN').clusterID.count().reset_index()
cellCount =  tMergeCluster.groupby(['ARFCN', 'PCI']).clusterID.nunique().reset_index()
cellCount[cellCount['ARFCN'] == 63600]

,ARFCN,PCI,clusterID


In [191]:
cellCount.groupby('ARFCN').clusterID.sum()

ARFCN
431570     14
625324     76
631000    123
636000      2
Name: clusterID, dtype: int64

In [169]:
summaryTable = pd.merge(uPCI, uPCIfRsrp,left_on='ARFCN', right_on='ARFCN', how='left')
summaryTable = pd.merge(summaryTable, cellCount, left_on='ARFCN', right_on='ARFCN', how='left')
summaryTable

,ARFCN,uPCI,uPCIfRSRP,clusterID
0,431570,15,15,14
1,625324,81,75,76
2,631000,137,123,123
3,636000,44,5,2


In [170]:
cellCount['clusterID'].sum()

215

In [171]:
#calculate the estimated lat lon on cell level which means on operatorBand, PCI, clusterID
tLocation = list()
for oprBand in tMergeCluster['ARFCN'].unique():
    #get unique PCI on oprBands level
    for tpci in tMergeCluster[tMergeCluster['ARFCN'] == oprBand]['PCI'].unique():
        tdf = tMergeCluster[(tMergeCluster['ARFCN'] == oprBand) & (tMergeCluster['PCI'] == tpci)]
        #get the estimated location of the cell location 
        for cluster in range((tdf['clusterID'].max())+1):
            lat = tdf[tdf['clusterID'] == cluster].sort_values(['RSRP'], ascending = False).head(5)['Longitude'].mean()
            lon = tdf[tdf['clusterID'] == cluster].sort_values(['RSRP'], ascending = False).head(5)['Latitude'].mean()
            d = {'ARFCN': oprBand, 'PCI': tpci, 'clusterID': cluster,'lon': lon, 'lat': lat}
            tLocation.append(pd.DataFrame(d, index=[0]))
            
tLocation = pd.concat(tLocation)

In [172]:
tLocation

,ARFCN,PCI,clusterID,lon,lat
0,431570,848.0,0,25.026128,121.063524
0,431570,12.0,0,24.997474,121.199422
0,431570,12.0,1,25.019118,121.220638
0,431570,479.0,0,25.077642,121.230356
0,431570,459.0,0,25.078388,121.231658
...,...,...,...,...,...
0,631000,164.0,0,24.997524,121.325082
0,631000,400.0,0,24.988358,121.325044
0,631000,266.0,0,25.017802,121.404362
0,636000,144.0,0,25.017278,121.402476


In [196]:
tLocation.groupby('ARFCN').count()

,PCI,clusterID,lon,lat
ARFCN,,,,
431570,14,14,14,14
625324,76,76,76,76
631000,123,123,123,123
636000,2,2,2,2
